In [1]:
!pip install --upgrade openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.0/662.0 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.2
    Uninstalling openai-1.76.2:
      Successfully uninstalled openai-1.76.2


In [2]:
from openai import OpenAI

In [ ]:
OPENAI_API_KEY = '....'  # Enter OpenAI secret key

client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
import os
import json
import csv
from openai import OpenAI

# Load your API key from environment variable
OPENAI_API_KEY = '...'  # Enter OpenAI secret key

client = OpenAI(api_key=OPENAI_API_KEY)

# Batch settings
BATCH_SIZE = 10

# System prompt for batched sentiment analysis
system_instruction = """
You are an expert sentiment analyst. You will be given a JSON array of review objects.
Each object has:
  - review_id: a unique identifier for the review
  - review_snippet: the first 50 characters of the review text
For each object, return a new object with keys:
  - review_id: same identifier
  - review_snippet: the original snippet
  - sentiment: one of \"Positive\", \"Neutral\", or \"Negative\"
  - score: an integer from 0 to 10 representing how positive it is
Return only a JSON array of these objects, without any additional text.
"""

# Helper to split list into batches
def chunk(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i + size]

# Load dataset
with open("sample_200_review.json", "r", encoding="utf-8") as f:
    reviews = json.load(f)

all_results = []
for batch_index, batch in enumerate(chunk(reviews, BATCH_SIZE), start=1):
    # Build list of dicts with review_id and snippet
    payload = [
        {"review_id": entry.get("review_id"),
         "review_snippet": entry.get("text", "")[:50]}
        for entry in batch
    ]
    user_message = json.dumps(payload, ensure_ascii=False)

    messages = [
        {"role": "system", "content": system_instruction},
        {"role": "user",   "content": user_message}
    ]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    raw = response.choices[0].message.content.strip()
    try:
        batch_results = json.loads(raw)
    except json.JSONDecodeError:
        # Attempt to extract JSON array from the response
        start = raw.find('[')
        end = raw.rfind(']')
        if start != -1 and end != -1:
            json_str = raw[start:end+1]
            try:
                batch_results = json.loads(json_str)
            except json.JSONDecodeError:
                print(f"Batch {batch_index}: Failed to parse JSON even after extraction. Raw response:\n{raw}")
                continue
        else:
            print(f"Batch {batch_index}: No JSON array found in response. Raw response:\n{raw}")
            continue

    all_results.extend(batch_results)
    print(f"Processed batch {batch_index}, got {len(batch_results)} results.")

# Write to CSV
csv_file = "sentiment_results.csv"
with open(csv_file, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["review_id", "review_snippet", "sentiment", "score"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for item in all_results:
        writer.writerow(item)

print(f"Sentiment analysis complete. Total items: {len(all_results)}. Results saved to {csv_file}.")


Processed batch 1, got 10 results.
Processed batch 2, got 10 results.
Processed batch 3, got 10 results.
Processed batch 4, got 10 results.
Processed batch 5, got 10 results.
Processed batch 6, got 10 results.
Processed batch 7, got 10 results.
Processed batch 8, got 10 results.
Processed batch 9, got 10 results.
Processed batch 10, got 10 results.
Processed batch 11, got 10 results.
Processed batch 12, got 10 results.
Processed batch 13, got 10 results.
Processed batch 14, got 10 results.
Processed batch 15, got 10 results.
Processed batch 16, got 10 results.
Processed batch 17, got 10 results.
Batch 18: No JSON array found in response. Raw response:
[{"review_id": "Q0FPQXpgmAVyAeStInrcuA", "review_snippet": "Awesome place.  Enormous selections of clubs/equip", "sentiment": "Positive", "score": 9}, {"review_id": "R6BSs5jmGIP1UlTFyYVWhw", "review_snippet": "Food: Wife had the grilled vegetable salad (pictur", "sent...
Processed batch 19, got 10 results.
Processed batch 20, got 10 resul